In [2]:
#Imports + Load Data

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

# Load raw dataset (upload first in Colab)
df = pd.read_csv("/content/drive/MyDrive/transport_raw.csv", parse_dates=["Date","ScheduledArrival","ActualArrival"])

df.head()


,Date,RouteID,StopID,ScheduledArrival,ActualArrival,DelayMinutes,PassengerCount,Capacity,LoadFactor,DistanceKM,Weather,AnomalyFlag
0,2024-02-26,R039,S0031,2024-02-26 09:29:00,2024-02-26 09:36:37.350096300,7.62,32,80,0.40,7.49,Rain,0
1,2024-01-25,R029,S0332,2024-01-25 20:17:00,2024-01-25 20:19:44.901191100,2.75,5,60,0.08,3.09,Rain,0
2,2024-02-17,R015,S0411,2024-02-17 19:10:00,2024-02-17 19:25:05.353291440,15.09,28,80,0.35,9.07,Clear,0
3,2024-03-10,R043,S0052,2024-03-10 04:00:00,2024-03-10 04:00:24.894476700,0.41,47,70,0.67,6.58,Clear,0
4,2024-02-11,R008,S0384,2024-02-11 00:27:00,2024-02-11 00:31:12.132600360,4.20,68,60,1.13,2.37,Clear,0


In [3]:
#Basic Data Inspection

print("INFO:")
display(df.info())

print("\nDESCRIPTION:")
display(df.describe(include='all'))

print("\nMISSING VALUE SUMMARY:")
display(df.isnull().sum())


INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              40000 non-null  datetime64[ns]
 1   RouteID           40000 non-null  object        
 2   StopID            40000 non-null  object        
 3   ScheduledArrival  40000 non-null  datetime64[ns]
 4   ActualArrival     40000 non-null  datetime64[ns]
 5   DelayMinutes      40000 non-null  float64       
 6   PassengerCount    40000 non-null  int64         
 7   Capacity          40000 non-null  int64         
 8   LoadFactor        40000 non-null  float64       
 9   DistanceKM        40000 non-null  float64       
 10  Weather           40000 non-null  object        
 11  AnomalyFlag       40000 non-null  int64         
dtypes: datetime64[ns](3), float64(3), int64(3), object(3)
memory usage: 3.7+ MB


None


DESCRIPTION:


,Date,RouteID,StopID,ScheduledArrival,ActualArrival,DelayMinutes,PassengerCount,Capacity,LoadFactor,DistanceKM,Weather,AnomalyFlag
count,40000,40000,40000,40000,40000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000,40000.000000
unique,NaN,50,600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN
top,NaN,R013,S0330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Clear,NaN
freq,NaN,889,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28048,NaN
mean,2024-02-29 12:29:33.360000,NaN,NaN,2024-02-29 23:31:36.516000,2024-02-29 23:37:26.047848960,7.049810,42.053975,65.016000,0.667071,6.269597,NaN,0.049175
min,2024-01-01 00:00:00,NaN,NaN,2024-01-01 00:09:00,2024-01-01 00:07:03.656219400,-5.000000,5.000000,50.000000,0.060000,0.500000,NaN,0.000000
25%,2024-01-30 00:00:00,NaN,NaN,2024-01-30 14:16:00,2024-01-30 14:25:18.688475136,-1.280000,23.000000,50.000000,0.360000,3.410000,NaN,0.000000
50%,2024-03-01 00:00:00,NaN,NaN,2024-03-01 00:59:30,2024-03-01 01:04:27.643412992,5.625000,42.000000,70.000000,0.650000,6.290000,NaN,0.000000
75%,2024-03-31 00:00:00,NaN,NaN,2024-03-31 12:27:15,2024-03-31 12:30:08.417702144,12.830000,61.000000,80.000000,0.940000,9.142500,NaN,0.000000
max,2024-04-29 00:00:00,NaN,NaN,2024-04-29 21:51:00,2024-04-29 22:04:25.065271200,77.640000,79.000000,80.000000,1.580000,12.000000,NaN,1.000000



MISSING VALUE SUMMARY:


,0
Date,0
RouteID,0
StopID,0
ScheduledArrival,0
ActualArrival,0
DelayMinutes,0
PassengerCount,0
Capacity,0
LoadFactor,0
DistanceKM,0


In [4]:
#Timestamp Validation & Recalculate Delay

# Calculate time difference
df['TimeDiff'] = (df['ActualArrival'] - df['ScheduledArrival']).dt.total_seconds() / 60

# Fix wrong negative delays (beyond tolerance)
df.loc[df['TimeDiff'] < -2, 'ActualArrival'] = df['ScheduledArrival']

# Recalculate delay
df['DelayMinutes'] = (df['ActualArrival'] - df['ScheduledArrival']).dt.total_seconds() / 60

df['DelayMinutes'] = df['DelayMinutes'].clip(lower=-1)  # allow small early arrivals
df.head()


,Date,RouteID,StopID,ScheduledArrival,ActualArrival,DelayMinutes,PassengerCount,Capacity,LoadFactor,DistanceKM,Weather,AnomalyFlag,TimeDiff
0,2024-02-26,R039,S0031,2024-02-26 09:29:00,2024-02-26 09:36:37.350096300,7.622502,32,80,0.40,7.49,Rain,0,7.622502
1,2024-01-25,R029,S0332,2024-01-25 20:17:00,2024-01-25 20:19:44.901191100,2.748353,5,60,0.08,3.09,Rain,0,2.748353
2,2024-02-17,R015,S0411,2024-02-17 19:10:00,2024-02-17 19:25:05.353291440,15.089222,28,80,0.35,9.07,Clear,0,15.089222
3,2024-03-10,R043,S0052,2024-03-10 04:00:00,2024-03-10 04:00:24.894476700,0.414908,47,70,0.67,6.58,Clear,0,0.414908
4,2024-02-11,R008,S0384,2024-02-11 00:27:00,2024-02-11 00:31:12.132600360,4.202210,68,60,1.13,2.37,Clear,0,4.202210


In [5]:
#Handle Missing Values

# Weather → forward fill
df['Weather'] = df['Weather'].fillna(method='ffill')

# Capacity → most frequent
df['Capacity'] = df['Capacity'].fillna(df['Capacity'].mode()[0])

# Passenger count → median
df['PassengerCount'] = df['PassengerCount'].fillna(df['PassengerCount'].median())

# Recalculate Load Factor if needed
df['LoadFactor'] = df['PassengerCount'] / df['Capacity']

df.isnull().sum()


/tmp/ipython-input-1204151845.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Weather'] = df['Weather'].fillna(method='ffill')


,0
Date,0
RouteID,0
StopID,0
ScheduledArrival,0
ActualArrival,0
DelayMinutes,0
PassengerCount,0
Capacity,0
LoadFactor,0
DistanceKM,0


In [6]:
#Outlier Removal (Z-score)

delay_mean = df['DelayMinutes'].mean()
delay_std = df['DelayMinutes'].std()

z = np.abs((df['DelayMinutes'] - delay_mean) / delay_std)
df = df[z < 3]

df.shape


(39681, 13)

In [7]:
#Feature Engineering

# TIME FEATURES
df['Hour'] = df['ScheduledArrival'].dt.hour
df['Minute'] = df['ScheduledArrival'].dt.minute
df['DayOfWeek'] = df['ScheduledArrival'].dt.day_name()
df['IsWeekend'] = df['DayOfWeek'].isin(['Saturday','Sunday']).astype(int)

# Peak hour flag
df['PeakHourFlag'] = df['Hour'].apply(lambda x: 1 if x in [7,8,9,17,18,19] else 0)

# Load category buckets
df['LoadCategory'] = pd.cut(
    df['LoadFactor'],
    bins=[0, 0.4, 0.7, 1.0],
    labels=['Low', 'Medium', 'High']
)

# Reliability Score
# 1 = on time, 0.5 = small delay, 0 = big delay
df['ReliabilityScore'] = df['DelayMinutes'].apply(
    lambda x: 1 if x <= 3 else (0.5 if x <= 10 else 0)
)

# Weather impact score
weather_map = {"Clear": 0, "Fog": 5, "Rain": 10, "Storm": 20}
df['WeatherImpact'] = df['Weather'].map(weather_map)

# Congestion Index
df['CongestionIndex'] = df['PassengerCount'] / df['Capacity']

# Delay Severity Category
df['DelaySeverity'] = pd.cut(
    df['DelayMinutes'],
    bins=[-5, 3, 10, 60],
    labels=["Early/On-time", "Moderate Delay", "Severe Delay"]
)

# Distance efficiency (Delay per km)
df['DelayPerKM'] = df['DelayMinutes'] / df['DistanceKM']

# Trip efficiency score
df['TripEfficiency'] = (
    (1 - df['CongestionIndex']) * 0.2 +
    (df['ReliabilityScore']) * 0.5 +
    (1 - (df['WeatherImpact']/20)) * 0.3
)

df.head()


,Date,RouteID,StopID,ScheduledArrival,ActualArrival,DelayMinutes,PassengerCount,Capacity,LoadFactor,DistanceKM,Weather,AnomalyFlag,TimeDiff,Hour,Minute,DayOfWeek,IsWeekend,PeakHourFlag,LoadCategory,ReliabilityScore,WeatherImpact,CongestionIndex,DelaySeverity,DelayPerKM,TripEfficiency
0,2024-02-26,R039,S0031,2024-02-26 09:29:00,2024-02-26 09:36:37.350096300,7.622502,32,80,0.400000,7.49,Rain,0,7.622502,9,29,Monday,0,1,Low,0.5,10,0.400000,Moderate Delay,1.017690,0.520000
1,2024-01-25,R029,S0332,2024-01-25 20:17:00,2024-01-25 20:19:44.901191100,2.748353,5,60,0.083333,3.09,Rain,0,2.748353,20,17,Thursday,0,0,Low,1.0,10,0.083333,Early/On-time,0.889435,0.833333
2,2024-02-17,R015,S0411,2024-02-17 19:10:00,2024-02-17 19:25:05.353291440,15.089222,28,80,0.350000,9.07,Clear,0,15.089222,19,10,Saturday,1,1,Low,0.0,0,0.350000,Severe Delay,1.663641,0.430000
3,2024-03-10,R043,S0052,2024-03-10 04:00:00,2024-03-10 04:00:24.894476700,0.414908,47,70,0.671429,6.58,Clear,0,0.414908,4,0,Sunday,1,0,Medium,1.0,0,0.671429,Early/On-time,0.063056,0.865714
4,2024-02-11,R008,S0384,2024-02-11 00:27:00,2024-02-11 00:31:12.132600360,4.202210,68,60,1.133333,2.37,Clear,0,4.202210,0,27,Sunday,1,0,NaN,0.5,0,1.133333,Moderate Delay,1.773084,0.523333


In [8]:
#Route Performance Scoring

route_perf = df.groupby("RouteID")["DelayMinutes"].mean().reset_index()
route_perf.columns = ["RouteID", "AvgRouteDelay"]

df = df.merge(route_perf, on="RouteID", how="left")

# Categorize route performance
df['RoutePerformanceTag'] = df['AvgRouteDelay'].apply(
    lambda x: "Excellent" if x < 3 else ("Moderate" if x < 10 else "Poor")
)

df.head()


,Date,RouteID,StopID,ScheduledArrival,ActualArrival,DelayMinutes,PassengerCount,Capacity,LoadFactor,DistanceKM,Weather,AnomalyFlag,TimeDiff,Hour,Minute,DayOfWeek,IsWeekend,PeakHourFlag,LoadCategory,ReliabilityScore,WeatherImpact,CongestionIndex,DelaySeverity,DelayPerKM,TripEfficiency,AvgRouteDelay,RoutePerformanceTag
0,2024-02-26,R039,S0031,2024-02-26 09:29:00,2024-02-26 09:36:37.350096300,7.622502,32,80,0.400000,7.49,Rain,0,7.622502,9,29,Monday,0,1,Low,0.5,10,0.400000,Moderate Delay,1.017690,0.520000,6.544181,Moderate
1,2024-01-25,R029,S0332,2024-01-25 20:17:00,2024-01-25 20:19:44.901191100,2.748353,5,60,0.083333,3.09,Rain,0,2.748353,20,17,Thursday,0,0,Low,1.0,10,0.083333,Early/On-time,0.889435,0.833333,6.746122,Moderate
2,2024-02-17,R015,S0411,2024-02-17 19:10:00,2024-02-17 19:25:05.353291440,15.089222,28,80,0.350000,9.07,Clear,0,15.089222,19,10,Saturday,1,1,Low,0.0,0,0.350000,Severe Delay,1.663641,0.430000,6.348326,Moderate
3,2024-03-10,R043,S0052,2024-03-10 04:00:00,2024-03-10 04:00:24.894476700,0.414908,47,70,0.671429,6.58,Clear,0,0.414908,4,0,Sunday,1,0,Medium,1.0,0,0.671429,Early/On-time,0.063056,0.865714,6.477806,Moderate
4,2024-02-11,R008,S0384,2024-02-11 00:27:00,2024-02-11 00:31:12.132600360,4.202210,68,60,1.133333,2.37,Clear,0,4.202210,0,27,Sunday,1,0,NaN,0.5,0,1.133333,Moderate Delay,1.773084,0.523333,7.072954,Moderate


In [9]:
#Save Cleaned Dataset

df.to_csv("transport_cleaned.csv", index=False)
df.head()


,Date,RouteID,StopID,ScheduledArrival,ActualArrival,DelayMinutes,PassengerCount,Capacity,LoadFactor,DistanceKM,Weather,AnomalyFlag,TimeDiff,Hour,Minute,DayOfWeek,IsWeekend,PeakHourFlag,LoadCategory,ReliabilityScore,WeatherImpact,CongestionIndex,DelaySeverity,DelayPerKM,TripEfficiency,AvgRouteDelay,RoutePerformanceTag
0,2024-02-26,R039,S0031,2024-02-26 09:29:00,2024-02-26 09:36:37.350096300,7.622502,32,80,0.400000,7.49,Rain,0,7.622502,9,29,Monday,0,1,Low,0.5,10,0.400000,Moderate Delay,1.017690,0.520000,6.544181,Moderate
1,2024-01-25,R029,S0332,2024-01-25 20:17:00,2024-01-25 20:19:44.901191100,2.748353,5,60,0.083333,3.09,Rain,0,2.748353,20,17,Thursday,0,0,Low,1.0,10,0.083333,Early/On-time,0.889435,0.833333,6.746122,Moderate
2,2024-02-17,R015,S0411,2024-02-17 19:10:00,2024-02-17 19:25:05.353291440,15.089222,28,80,0.350000,9.07,Clear,0,15.089222,19,10,Saturday,1,1,Low,0.0,0,0.350000,Severe Delay,1.663641,0.430000,6.348326,Moderate
3,2024-03-10,R043,S0052,2024-03-10 04:00:00,2024-03-10 04:00:24.894476700,0.414908,47,70,0.671429,6.58,Clear,0,0.414908,4,0,Sunday,1,0,Medium,1.0,0,0.671429,Early/On-time,0.063056,0.865714,6.477806,Moderate
4,2024-02-11,R008,S0384,2024-02-11 00:27:00,2024-02-11 00:31:12.132600360,4.202210,68,60,1.133333,2.37,Clear,0,4.202210,0,27,Sunday,1,0,NaN,0.5,0,1.133333,Moderate Delay,1.773084,0.523333,7.072954,Moderate
